In [1]:
import pandas as pd
from pathlib import Path
import numpy as np

In [2]:
cwd = Path.cwd()
data_path = cwd.parent/'raw_data'
out_path = cwd.parent/'cleaned_data'

if not out_path.is_dir():
    out_path.mkdir()

In [3]:
participant_ids = []
for i in data_path.iterdir():
    if '.csv' in i.name:
        #print(i.name,i.name.split('_'))
        participant_ids.append(i.name.split('_')[1])
unique_pid = np.unique(participant_ids)

In [4]:
unique_pid

array(['1', '10', '11', '12', '13', '14', '16', '17', '18', '19', '2',
       '20', '21', '22', '23', '24', '25', '26', '29', '3', '31', '32',
       '33', '4', '5', '6', '7', '8', '9'], dtype='<U2')

In [5]:
for i in data_path.iterdir():
    if '.csv' in i.name:
        print(i)

/Users/josephpruitt/Desktop/exp3/raw_data/exp3_7_A.csv
/Users/josephpruitt/Desktop/exp3/raw_data/exp3_18_A.csv
/Users/josephpruitt/Desktop/exp3/raw_data/exp3_5_A.csv
/Users/josephpruitt/Desktop/exp3/raw_data/exp3_25_A.csv
/Users/josephpruitt/Desktop/exp3/raw_data/exp3_21_A.csv
/Users/josephpruitt/Desktop/exp3/raw_data/exp3_1_A.csv
/Users/josephpruitt/Desktop/exp3/raw_data/exp3_3_A.csv
/Users/josephpruitt/Desktop/exp3/raw_data/exp3_23_A.csv
/Users/josephpruitt/Desktop/exp3/raw_data/exp3_4_A.csv
/Users/josephpruitt/Desktop/exp3/raw_data/exp3_24_A.csv
/Users/josephpruitt/Desktop/exp3/raw_data/exp3_19_A.csv
/Users/josephpruitt/Desktop/exp3/raw_data/exp3_26_A.csv
/Users/josephpruitt/Desktop/exp3/raw_data/exp3_6_A.csv
/Users/josephpruitt/Desktop/exp3/raw_data/exp3_2_A.csv
/Users/josephpruitt/Desktop/exp3/raw_data/exp3_22_A.csv
/Users/josephpruitt/Desktop/exp3/raw_data/exp3_20_A.csv
/Users/josephpruitt/Desktop/exp3/raw_data/exp3_11_A.csv
/Users/josephpruitt/Desktop/exp3/raw_data/exp3_13_A.csv

In [7]:
cols = ['participant','position', 'validity', 'Stimulus', 'response_key', 'response_rt', 'response_correct', 'confidence','confidence_rt']

all_data = pd.DataFrame()
map_degrees ={-557:'10', 557:'10', -1158:'20',1158:'20',1716:'30',
              -1716:'30',2274:'40',-2274:'40'}

map_degrees2 = {-557:'-10', 557:'10', -1158:'-20',1158:'20',1716:'30',
              -1716:'-30',2274:'40',-2274:'-40'}

for pid in unique_pid:
    
    df = pd.read_csv(data_path/f'exp3_{pid}_A.csv')
    
    df = df[cols] 
    
    df = df.loc[~df.Stimulus.isnull()]
    df['degrees'] = df['position'].map(map_degrees)
    df['degrees_uncollapsed'] = df['position'].map(map_degrees2)
    df.reset_index(inplace=True)
    df.drop(columns = 'index',inplace=True)
    df.index.name = 'trials'
    df.reset_index(inplace=True)
    
    #Need to account for the fact that the first 225 rows are the staircase
    df['trials'] = df.trials+1
    df.rename(columns={'trials':'trialsN'},inplace=True)
    
    df['trialsPos'] = df.groupby(['participant','degrees']).cumcount()+1
    
    df["response_key"].replace({2.0: "0", '2':"0"}, inplace=True)
    
    all_data = pd.concat([all_data,df],ignore_index=True)
    
    df.to_csv(out_path/f'exp3_{pid}_cleaned.csv',index=False)
    
all_data.to_csv(out_path/'all_pid_exp3_clean.csv',index=False)

In [19]:
# Threshold File

cols = ['participant','trials.label','trials.cue', 'trials.thisRepN', 'trials.thisN', 'signalOpacity', 'trials.response']

stair_data = pd.DataFrame()

for pid in unique_pid:
    
    df1 = pd.read_csv(data_path/f'exp3_{pid}_A.csv')
    
    df1 = df1[cols] 
    
    df1 = df1.loc[~df1.signalOpacity.isnull()]
    df1.reset_index(inplace=True)
    df1.drop(columns = 'index',inplace=True)
    df1.index.name = 'trials'
    df1.reset_index(inplace=True)
    
    #Need to account for the fact that the first 225 rows are the staircase
    df1['trialsStair'] = df1.groupby(['participant','trials.label']).cumcount()+1

    df1.rename(columns={'trials':'trialsN'},inplace=True)
    
    stair_data = pd.concat([stair_data,df1],ignore_index=True)
    
stair_data.to_csv(out_path/'all_pid_exp3_staircase.csv',index=False)